In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%%capture
!pip install --upgrade tensorflow
!pip show tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import csv
import re
import cv2
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import Huber
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse_met
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.utils import shuffle
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam
import tensorflow.keras
from tensorflow.python.keras import backend as k
from tensorflow.keras.models import Model
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import (ModelCheckpoint,Callback,LearningRateScheduler,EarlyStopping,ReduceLROnPlateau,CSVLogger)
from imgaug import augmenters as iaa
import imgaug as ia
import warnings
warnings.filterwarnings("ignore")
CLASSS = {0: "No DR", 1: "Mild", 2: "Moderate", 3: "Severe", 4: "Proliferative DR"}

# ovde zameni path sa svojim path u blindness Challenge folder, glavni folder
dest_only_2019 = "/content/drive/My Drive/Colab Notebooks/BlindnessChallenge"

IMG_SIZE = 512
EPOCHS = 30
batch_size = 4
CLASS_ID = 4
WORKERS = 2
# i ovde kod checkpoint obavezno zameni path samo da bude tvoj_path/working/nevena_ep_512_{epoch:02d}.hdf5
checkpoint = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/BlindnessChallenge/working/nevena_ep_512_{epoch:02d}.hdf5", monitor="val_loss", verbose=1,
                             save_best_only=False, mode='min', save_weights_only=False)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, mode='auto',
                                   min_delta=0.0001)
early = EarlyStopping(monitor="val_loss", mode="min", patience=9)
csv_logger = CSVLogger(filename=f'{dest_only_2019}/working/nevena_training_log_512.csv', separator=',', append=True)

In [ ]:
def loadDatasets(dest):
    df_train_2019 = pd.read_csv(f"{dest}/trainLabels19.csv")
    df_train_2015 = pd.read_csv(f"{dest}/trainLabels15.csv")
    df_train_2015.columns = df_train_2019.columns
    #print(df_train_2015.head())
    df_train = pd.concat([df_train_2015,df_train_2019],axis=0)
    x,y = shuffle(df_train['id_code'],df_train['diagnosis'],random_state=42)
    #y = to_categorical(y, num_classes=5)
    #x_val,y = shuffle(df_test['id_code'],df_test['diagnosis'],random_state=42)
    #y_val = to_categorical(y, num_classes=5)
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15,
                                                      stratify=y, random_state=42)
    return x_train, x_val, y_train, y_val

def loadDatasets_only_2019(dest_only_2019):
    df_train = pd.read_csv(f"{dest_only_2019}/train.csv")
    train_aug = pd.read_csv(f"{dest_only_2019}/augmentacije.csv")
    train_aug.drop(train_aug.columns[0],axis=1,inplace=True)
    train_aug.index = [i for i in range(3662,3662+838)]
    train_aug.columns = ['id_code','diagnosis']

    df_train = pd.concat([df_train,train_aug])
    #print(df_train.sample)
    x,y = shuffle(df_train['id_code'],df_train['diagnosis'],random_state=42)
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15,
                                                      stratify=y, random_state=42)
    return x_train, x_val, y_train, y_val
    
x_train, x_val, y_train, y_val = loadDatasets_only_2019(dest_only_2019)
#train_aug = pd.read_csv(f"{dest_only_2019}/augmentacije.csv")
#x_train = pd.concat([x_train,train_aug['0']])
#y_train = pd.concat([y_train,train_aug['1']])
y_train = to_categorical(y_train, num_classes=5)
y_val = to_categorical(y_val, num_classes=5)


#preprocessing
#mean=(0,485, 0,456, 0,406) and std=(0,229, 0,224, 0,225)
def normalization_fn(img):
    img = np.divide(img,255)
    mean_value  = [0.485,0.456,0.406]
    std_value = [0.229,0.224,0.225]
    std_after = []
    mean_after = []
    for i in range(3):
        norm_tmp = np.subtract(img[:,:,i],np.mean(img[:,:,i]))
        std_mult = np.divide(std_value[i],np.std(img[:,:,i], dtype=np.float64))
        img[:,:,i] = np.add(mean_value[i],np.multiply(norm_tmp,std_mult))
        #mean_after.append(np.mean(img[:,:,i]))
        #std_after.append(np.std(img[:,:,i]))
    return img

In [ ]:
def bens_processing_plot(df_train,dest,IMG_SIZE):
    sigmaX = 50
    fig,ax = plt.subplots(2, 6, figsize=(15, 6))
    df_tmp = df_train.sample(12)
    for idxn,(id,row) in enumerate(df_tmp.iteritems()):
        #print(row)
        path= f"{dest}/train_images/{row}.png"
        # image = cv2.imread(path)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        # image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , IMG_SIZE/10) ,-4 ,128)
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = crop_image_from_gray(image)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        img=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        #privremeno da vidim kako radi seq_boris i konstruisem svoj filter, kasnije obrisati ili zakomentarisati
        #img = seq.augment_image(img)

        row = idxn // 6
        col = idxn % 6
        ax[row,col].imshow(img)
    #plt.show()
    #return image
def bens_processing(img,IMG_SIZE):
    sigma = 50
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    image = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigma) ,-4 ,128)
    return image

    
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq_boris = iaa.Sequential(
    [
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2),
        sometimes(iaa.Affine(
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)}, # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
                rotate=(-10, 10), # rotate by -45 to +45 degrees
                shear=(-5, 5), # shear by -16 to +16 degrees
                order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 255), # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
        iaa.SomeOf((0, 4),
                [
                    sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(10, 50))), # convert images into their superpixel representation
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 1.0)), # blur images with a sigma between 0 and 3.0
                        iaa.AverageBlur(k=(3, 5)), # blur image using local means with kernel sizes between 2 and 7
                        iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)), # sharpen images
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                    # search either for all edges or for directed edges,
                    # blend the result with the original image using a blobby mask
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5), # add gaussian noise to images
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.05), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
                    ]),
                    iaa.Invert(0.01, per_channel=True), # invert color channels
                    iaa.Add((-2, 2), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-1, 1)), # change hue and saturation
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.OneOf([
                        iaa.Multiply((0.9, 1.1), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-1, 0),
                            first=iaa.Multiply((0.9, 1.1), per_channel=True),
                            second=iaa.ContrastNormalization((0.9, 1.1))
                        )
                    ])
                ],
                random_order=True
            )
        
        
    ],random_order=True)

In [ ]:
def create_model(input_shape, n_out,dest):
    input_tensor = Input(shape=input_shape)
    base_model = ResNet50(include_top=False,
                   weights='imagenet',
                   input_tensor=input_tensor)
    #base_model=load_model("/content/drive/My Drive/Colab Notebooks/BlindnessChallenge/efficient_net/working/weights_ep0_batch_1950.h5")
    #model = base_model
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(5, name='final_output')(x)
    model = Model(input_tensor, final_output)
    return model

model = create_model(
        input_shape=(IMG_SIZE,IMG_SIZE, 3),
        n_out=5,dest=dest_only_2019)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
class My_Generator(Sequence):
    def __init__(self,image_filenames,labels,dest,
            batch_size,is_train=True,
            mix=False,augment=False,IMG_size=512):
        self.image_filenames,self.labels = image_filenames,labels
        self.batch_size = batch_size
        self.IMG_size = IMG_size
        self.dest = dest
        self.is_train = is_train
        self.is_augment = augment
        if(self.is_train):
            self.on_epoch_end()
        self.is_mix = mix
        
    def __len__(self):
        return int(np.ceil(len(self.image_filenames)/float(self.batch_size)))
    
    def __getitem__(self,idx):
        batch_x = self.image_filenames[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_y = self.labels[idx*self.batch_size:(idx+1)*self.batch_size]
        
        if(self.is_train):
            return self.train_generate(batch_x,batch_y)
        return self.valid_generate(batch_x,batch_y)
        
    def on_epoch_end(self):
        if(self.is_train):
            self.image_filenames,self.labels = shuffle(self.image_filenames,self.labels)
        else:
            pass
    
    def mix_up(self,x,y):
        lam = np.random.beta(0.2,0.4)
        ori_index = np.arange(int(len(x)))
        index_array = np.arange(int(len(x)))
        np.random.shuffle(index_array)
        
        mixed_x = lam*x[ori_index] + (1-lam)*x[index_array]
        mixed_y = lam*y[ori_index] + (1-lam)*y[index_array]
        
        return mixed_x, mixed_y

    def train_generate(self,batch_x,batch_y):
        batch_images = []
        for (sample,label) in zip(batch_x,batch_y):
            if "augmented" in str(sample): 
                path=f"{dest_only_2019}/train_images_augmented/{sample}.png"
            else:
                path=f"{dest_only_2019}/train_images/{sample}.png"
            img = cv2.imread(path)
            img = bens_processing(img,self.IMG_size)
            img = cv2.resize(img,(self.IMG_size,self.IMG_size))
            if(self.is_augment):
                img = seq_boris.augment_image(img)
            img = normalization_fn(img)
            batch_images.append(img)
        batch_images = np.array(batch_images,np.float32)/255
        batch_y = np.array(batch_y,np.float32)
        if(self.is_mix):
            batch_images, batch_y = self.mix_up(batch_images, batch_y)
        return batch_images, batch_y
        
    def valid_generate(self, batch_x, batch_y):
        batch_images =[]
        for (sample,label) in zip(batch_x,batch_y):
            #print(sample)
            if "augmented" in str(sample): 
                path=f"{dest_only_2019}/train_images_augmented/{sample}.png"
            else:
                path=f"{dest_only_2019}/train_images/{sample}.png"
            img = cv2.imread(path)
            img = bens_processing(img,self.IMG_size)
            img = cv2.resize(img,(self.IMG_size,self.IMG_size))
            img = normalization_fn(img)
            batch_images.append(img)
        batch_images = np.array(batch_images, np.float32) / 255
        batch_y = np.array(batch_y, np.float32)
        return batch_images, batch_y
    
train_generator = My_Generator(x_train, y_train,dest_only_2019, batch_size, is_train=True)
train_mixup = My_Generator(x_train, y_train, dest_only_2019,batch_size, is_train=True, mix=False, augment=False)
valid_generator = My_Generator(x_val, y_val, dest_only_2019,batch_size, is_train=False)

def flatten_fn(y):
    for i in range(len(y)):
        if y[i] < 0.7:
            y[i] = 0
        elif y[i]> 0.7 and y[i]<1.5:
            y[i] = 1
        elif y[i]> 1.5 and y[i]< 2.5:
            y[i] = 2
        elif y[i] >2.5 and y[i]<3.5:
            y[i] = 3
        elif y[i]> 3.5:
            y[i] = 4
    return y


class QWKEvaluation(Callback):
    def __init__(self,validation_data=(),batch_size=32,interval=1):
        super(Callback,self).__init__()
        
        self.interval = interval
        self.batch_size = batch_size
        self.valid_generator, self.y_val = validation_data
        self.history = []
        self.batch_n = 100
    def on_epoch_begin(self,epoch,logs={}):
        #print(f'starting epoch: {epoch}')
        self.epoch = epoch
    #def on_batch_end(self, batch, logs={}):
    #    if batch % self.batch_n == 500 and batch > 1:
    #        name = f'/content/drive/My Drive/Colab Notebooks/BlindnessChallenge/working/nevena_weights_ep{self.epoch}_batch_{batch}.h5'
    #        self.model.save(name)
    '''
    def on_train_batch_begin(self,batch, logs=None):
        self.start_time = time.time()
    def on_train_batch_end(self,batch, logs=None):
        self.end_time = time.time()
        print(self.end_time - self.start_time)'''
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(generator=self.valid_generator, steps=np.ceil(float(len(self.y_val))/float(self.batch_size)),workers=1,use_multiprocessing=False,verbose=1)
            def flatten_class(y):
                return np.argmax(y, axis=1).reshape(-1)
            #y_tmp = self.y_val.to_numpy()

            score = cohen_kappa_score(flatten_class(y_val), flatten_class(y_pred), labels = [0,1,2,3,4],weights='quadratic')
            print(f"\n epoch: {epoch+1} - QWK_score: {score}")
            self.history.append(score)
            if score >= max(self.history):
                print(f"saving checkpoint:{score}")
                self.model.save("/content/drive/My Drive/Colab Notebooks/BlindnessChallenge/working/nevena_best_qwk.h5")
                
qwk = QWKEvaluation(validation_data=(valid_generator, y_val), batch_size=batch_size, interval=1)

In [ ]:
#i ovde obavezno zameni path kod checkpoint warm_up
checkpoint_warm_up = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/BlindnessChallenge/working/nevena_ep_512_warm_up_{epoch:02d}.hdf5", monitor="val_loss", verbose=1,
                             save_best_only=False, mode='min', save_weights_only=False)
for layer in model.layers:
    layer.trainable = False

for i in range(-3, 0):
    model.layers[i].trainable = True

model.compile(
    loss= 'categorical_crossentropy',
    optimizer=Adam(1e-3))

model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(y_train)) / float(batch_size)),
    epochs=2,
    workers=WORKERS, use_multiprocessing=True,
    verbose=1,
    callbacks=[qwk,checkpoint_warm_up])#moze true multiprocessing np.ceil(float(len(y_train)) / float(batch_size))

Epoch 1/2
 18/988 [..............................] - ETA: 5:42:48 - loss: 3.3059

Process Keras_worker_ForkPoolWorker-12:
Process Keras_worker_ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing

KeyboardInterrupt: ignored

In [ ]:

# ovde load model kad zavrsis dve epohe warm upa koje si radila ili kada ti tokom ove normalne obuke pukne na nekoj epohi recimo load model nevena_ep_13 ako ti je pukao na 14toj epohi i stavis onda initial epoch 13
# kad pocinjes nakon zavrsenog warm up onda idi load_model nevena_ep_512_warm_up_02.hdf5
callbacks_list = [checkpoint, reduceLROnPlat, early, qwk,csv_logger]
for layer in model.layers:
        layer.trainable = True

model.compile(loss='categorical_crossentropy',
              # loss=kappa_loss,
              optimizer=Adam(lr=1e-4))
model.fit_generator(
    train_mixup,
    steps_per_epoch=int(np.ceil(float(len(x_train)) / float(batch_size))),
    validation_data=valid_generator,
    validation_steps=int(np.ceil(float(len(x_val)) / float(batch_size))),
    epochs=30,
    verbose=1,max_queue_size=1,
    workers=1, use_multiprocessing=False,
    callbacks=callbacks_list,initial_epoch=0)
